# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

C:\Users\mleroi\AppData\Local\anaconda3\envs\learn-env\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\mleroi\AppData\Local\anaconda3\envs\learn-env\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\mleroi\AppData\Local\anaconda3\envs\learn-env\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = X_train_numeric.join(X_train_cat)
X_val = X_val_numeric.join(X_val_cat)
X_test = X_test_numeric.join(X_test_cat)

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [6]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()
baseline_model.add(layers.Dense(10, input_shape=n_features))

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu'))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(100, activation='relu'))


# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.keys() 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [23]:
# Train the model
normalized_input_model.fit(X_train,
                    y_train,
                    epochs=150,
                    batch_size=32,
                    validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [19]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [24]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train,
                    y_train_scaled,
                    epochs=150,
                    batch_size=32,
                    validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.3927 - mse: 0.3927 - val_loss: 0.2101 - val_mse: 0.2101
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2426 - mse: 0.2426 - val_loss: 0.1662 - val_mse: 0.1662
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1960 - mse: 0.1960 - val_loss: 0.1543 - val_mse: 0.1543
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1732 - mse: 0.1732 - val_loss: 0.1472 - val_mse: 0.1472
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1559 - mse: 0.1559 - val_loss: 0.1368 - val_mse: 0.1368
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1403 - mse: 0.1403 - val_loss: 0.1312 - val_mse: 0.1312
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1253 - mse: 0.1253 - val_loss: 0.1268 - val_mse: 0.1268
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1171 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.1273 - val_mse: 0.1273
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0218 - mse: 0.0218 - val_loss: 0.1282 - val_mse: 0.1282
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0216 - mse: 0.0216 - val_loss: 0.1282 - val_mse: 0.1282
Epoch 66/150
33/33 [==============================] - 0s 968us/step - loss: 0.0211 - mse: 0.0211 - val_loss: 0.1294 - val_mse: 0.1294
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0206 - mse: 0.0206 - val_loss: 0.1281 - val_mse: 0.1281
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0206 - mse: 0.0206 - val_loss: 0.1277 - val_mse: 0.1277
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0202 - mse: 0.0202 - val_loss: 0.1288 - val_mse: 0.1288
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0197 - mse:

33/33 [==============================] - 0s 1ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1328 - val_mse: 0.1328
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.1328 - val_mse: 0.1328
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.1318 - val_mse: 0.1318
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.1318 - val_mse: 0.1318
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.1316 - val_mse: 0.1316
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0087 - mse: 0.0087 - val_loss: 0.1318 - val_mse: 0.1318
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0089 - mse: 0.0089 - val_loss: 0.1311 - val_mse: 0.1311
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0086 -

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on training data
normalized_model.evaluate(X_train,y_train_scaled)

33/33 [==============================] - 0s 698us/step - loss: 0.0067 - mse: 0.0067


[0.006705042440444231, 0.006705042440444231]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [27]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val,y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1325 - mse: 0.1325


[0.13247551023960114, 0.13247551023960114]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [29]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

28602.19168110587

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [32]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape=n_features, kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
he_model.fit(X_train,
                    y_train_scaled,
                    epochs=150,
                    batch_size=32,
                    validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: 0.4871 - mse: 0.4871 - val_loss: 0.2255 - val_mse: 0.2255
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2871 - mse: 0.2871 - val_loss: 0.1796 - val_mse: 0.1796
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2168 - mse: 0.2168 - val_loss: 0.1544 - val_mse: 0.1544
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1888 - mse: 0.1888 - val_loss: 0.1398 - val_mse: 0.1398
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1636 - mse: 0.1636 - val_loss: 0.1264 - val_mse: 0.1264
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1509 - mse: 0.1509 - val_loss: 0.1478 - val_mse: 0.1478
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1434 - mse: 0.1434 - val_loss: 0.1261 - val_mse: 0.1261
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1270 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0251 - mse: 0.0251 - val_loss: 0.1235 - val_mse: 0.1235
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0248 - mse: 0.0248 - val_loss: 0.1238 - val_mse: 0.1238
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1226 - val_mse: 0.1226
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1229 - val_mse: 0.1229
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0235 - mse: 0.0235 - val_loss: 0.1234 - val_mse: 0.1234
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.1234 - val_mse: 0.1234
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0226 - mse: 0.0226 - val_loss: 0.1226 - val_mse: 0.1226
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0226 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1245 - val_mse: 0.1245
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1252 - val_mse: 0.1252
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1249 - val_mse: 0.1249
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1246 - val_mse: 0.1246
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1234 - val_mse: 0.1234
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1239 - val_mse: 0.1239
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1240 - val_mse: 0.1240
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [33]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 778us/step - loss: 0.0092 - mse: 0.0092


[0.009194496087729931, 0.009194496087729931]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [34]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1279 - mse: 0.1279


[0.1278630644083023, 0.1278630644083023]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [36]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.5403 - mse: 0.5403 - val_loss: 0.2489 - val_mse: 0.2489
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2478 - mse: 0.2478 - val_loss: 0.1849 - val_mse: 0.1849
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2027 - mse: 0.2027 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1654 - mse: 0.1654 - val_loss: 0.1504 - val_mse: 0.1504
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1488 - mse: 0.1488 - val_loss: 0.1453 - val_mse: 0.1453
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1331 - mse: 0.1331 - val_loss: 0.1408 - val_mse: 0.1408
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1209 - mse: 0.1209 - val_loss: 0.1463 - val_mse: 0.1463
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1140 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0249 - mse: 0.0249 - val_loss: 0.1704 - val_mse: 0.1704
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0242 - mse: 0.0242 - val_loss: 0.1730 - val_mse: 0.1730
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1716 - val_mse: 0.1716
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1694 - val_mse: 0.1694
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1771 - val_mse: 0.1771
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1679 - val_mse: 0.1679
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1720 - val_mse: 0.1720
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0222 - mse: 0

33/33 [==============================] - 0s 1ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1884 - val_mse: 0.1884
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.1925 - val_mse: 0.1925
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1900 - val_mse: 0.1900
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1922 - val_mse: 0.1922
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1909 - val_mse: 0.1909
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1912 - val_mse: 0.1912
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1919 - val_mse: 0.1919
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0105 -

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [37]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)
# lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1926 - mse: 0.1926


[0.1925562024116516, 0.1925562024116516]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [38]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1926 - mse: 0.1926


[0.1925562024116516, 0.1925562024116516]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [39]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                         loss='mse', 
                         metrics=['mse']) 
# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3700 - mse: 0.3700 - val_loss: 0.1939 - val_mse: 0.1939
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1844 - mse: 0.1844 - val_loss: 0.1417 - val_mse: 0.1417
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1380 - mse: 0.1380 - val_loss: 0.1935 - val_mse: 0.1935
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1093 - mse: 0.1093 - val_loss: 0.1094 - val_mse: 0.1094
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0772 - mse: 0.0772 - val_loss: 0.1423 - val_mse: 0.1423
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0771 - mse: 0.0771 - val_loss: 0.1116 - val_mse: 0.1116
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0549 - mse: 0.0549 - val_loss: 0.1523 - val_mse: 0.1523
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0465 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0082 - mse: 0.0082 - val_loss: 0.1195 - val_mse: 0.1195
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1202 - val_mse: 0.1202
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1152 - val_mse: 0.1152
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1236 - val_mse: 0.1236
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0086 - mse: 0.0086 - val_loss: 0.1208 - val_mse: 0.1208
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1131 - val_mse: 0.1131
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0061 - mse: 0.0061 - val_loss: 0.1255 - val_mse: 0.1255
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0085 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.1102 - val_mse: 0.1102
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0079 - mse: 0.0079 - val_loss: 0.1188 - val_mse: 0.1188
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0066 - mse: 0.0066 - val_loss: 0.1109 - val_mse: 0.1109
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.1087 - val_mse: 0.1087
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0060 - mse: 0.0060 - val_loss: 0.1082 - val_mse: 0.1082
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0054 - mse: 0.0054 - val_loss: 0.1113 - val_mse: 0.1113
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0050 - mse: 0.0050 - val_loss: 0.1161 - val_mse: 0.1161
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [40]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)
# lecun_model.evaluate(X_val, y_val_scaled)

33/33 [==============================] - 0s 672us/step - loss: 0.0096 - mse: 0.0096


[0.009602352976799011, 0.009602352976799011]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [41]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 932us/step - loss: 0.1179 - mse: 0.1179


[0.11790045350790024, 0.11790045350790024]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [42]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='adam', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.4416 - mse: 0.4416 - val_loss: 0.1620 - val_mse: 0.1620
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1772 - mse: 0.1772 - val_loss: 0.0985 - val_mse: 0.0985
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1154 - mse: 0.1154 - val_loss: 0.1095 - val_mse: 0.1095
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0824 - mse: 0.0824 - val_loss: 0.0901 - val_mse: 0.0901
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0682 - mse: 0.0682 - val_loss: 0.1007 - val_mse: 0.1007
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0511 - mse: 0.0511 - val_loss: 0.1168 - val_mse: 0.1168
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0427 - mse: 0.0427 - val_loss: 0.1110 - val_mse: 0.1110
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0336 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.1050 - val_mse: 0.1050
Epoch 63/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.1142 - val_mse: 0.1142
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0025 - mse: 0.0025 - val_loss: 0.1030 - val_mse: 0.1030
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0035 - mse: 0.0035 - val_loss: 0.1121 - val_mse: 0.1121
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0039 - mse: 0.0039 - val_loss: 0.1036 - val_mse: 0.1036
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0049 - mse: 0.0049 - val_loss: 0.1073 - val_mse: 0.1073
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0056 - mse: 0.0056 - val_loss: 0.1069 - val_mse: 0.1069
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0052 - mse: 0

33/33 [==============================] - 0s 1ms/step - loss: 3.4836e-04 - mse: 3.4836e-04 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 124/150
33/33 [==============================] - 0s 2ms/step - loss: 3.0851e-04 - mse: 3.0851e-04 - val_loss: 0.1037 - val_mse: 0.1037
Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 2.9890e-04 - mse: 2.9890e-04 - val_loss: 0.1080 - val_mse: 0.1080
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 3.8641e-04 - mse: 3.8641e-04 - val_loss: 0.1037 - val_mse: 0.1037
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 4.8030e-04 - mse: 4.8030e-04 - val_loss: 0.1059 - val_mse: 0.1059
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 6.3980e-04 - mse: 6.3980e-04 - val_loss: 0.1058 - val_mse: 0.1058
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 6.9586e-04 - mse: 6.9586e-04 - val_loss: 0.1071 - val_mse: 0.1071
Epoch 130/150
33/33 [======

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [44]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)
# lecun_model.evaluate(X_val, y_val_scaled)

33/33 [==============================] - 0s 931us/step - loss: 0.0019 - mse: 0.0019


[0.0018955410923808813, 0.0018955410923808813]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [45]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 778us/step - loss: 0.0957 - mse: 0.0957


[0.09572712332010269, 0.09572712332010269]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [46]:
# Evaluate the best model on test data
adam_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 965us/step - loss: 0.1717 - mse: 0.1717


[0.17169146239757538, 0.17169146239757538]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [47]:
# Generate predictions on test data
y_test_pred_scaled = adam_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

32561.607797707165

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.